In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
from datetime import date 
import json

# setting up match day 
t = date.today() 
today = t.strftime('%d/%m/%Y')
today



'06/12/2020'

In [2]:
class PremireLeague:
    
    def __init__(self, x):
        
        self.x = x
        
        self.link = 'https://www.premierleague.com/match/{}'.format(self.x)

        self.r = requests.get(self.link)

        self.soup = bs(self.r.content, 'html.parser')
        


In [3]:
    

######## home team ###############
    
class HomeTeam(PremireLeague):
    
    def __init__(self, x):
        
        super().__init__(x)

    def home_goal(self):

        H_Goal = []

        goals = self.soup.select('div.matchEvents div.home div.event')

        for goal in goals:

            name = goal.a.text


            try:
                action = goal.select('div.icnContainer span')[0].text

                name = action + ' - ' + name

                H_Goal.append(name)

            except IndexError:

                break

        return H_Goal
    
    def goal_no_home(self):
        
        goal_no_home = []
        
        goals = self.soup.select('div.matchEvents div.home div.event')
        
        pattern = re.compile(r"\d+") # to get time of goal
        
        for goal in goals:
            
            matches = pattern.findall(goal.text)

            goal_no_home.append(len(matches))
        
        return goal_no_home
        
# Home team assists

    def home_assist(self):

        H_Assist = []

        assists = self.soup.select('div.assists div.home div.event')

        for assist in assists:

            name = assist.a.text

            H_Assist.append(name)

        return H_Assist

# Home team name

    def home_name(self):

        home_name = self.soup.select('div.home')[0].select('span.long')[0].text

        return home_name
        
    def home(self):

        print(self.home_name() + ':- \n')

        print(f'{self.home_goal()} \n')

        print("Assist Given By:- \n")

        print(f'{self.home_assist()} \n')



In [4]:

########### Away Team #########


class AwayTeam(PremireLeague):
    
    def __init__(self, x):
        
        super().__init__(x)

#  Goal scored away team

    def away_goal(self):
        
        A_Goal = []

        goals = self.soup.select('div.matchEvents div.away div.event')

        for goal in goals:

            name = goal.a.text

            try:
                action = goal.select('div.icnContainer span')[0].text

                name = action + ' - ' + name

                A_Goal.append(name)

            except IndexError:

                break
                
        return A_Goal

# no of goals per player of away team 

    def goal_no_away(self):
        
        goal_no_away = []
        
        goals = self.soup.select('div.matchEvents div.away div.event')
        
        pattern = re.compile(r"\d+") # to get time of goal
        
        for goal in goals:
            
            matches = pattern.findall(goal.text)

            goal_no_away.append(len(matches))
        
        return goal_no_away


# Away team assists

    def away_assist(self):
        
        A_Assist = []

        assists = self.soup.select('div.assists div.away div.event')

        for assist in assists:

            name = assist.a.text

            A_Assist.append(name)

        return A_Assist
    
# Away team name

    def away_name(self):

        away_name = self.soup.select('div.away')[0].select('span.long')[0].text

        return away_name 

    def away(self):
        
        
        print(self.away_name() + ':- \n')

        print(f'{self.away_goal()} \n')

        print("Assist Given By:- \n")

        print(f'{self.away_assist()} \n')
    

In [5]:

############## Result#####################


class MatchResult(HomeTeam, AwayTeam):
    
    def __init__(self, x):
        
        super().__init__(x)
        
    def result(self):
        
        try:
            goal = self.soup.select('div.score')[0]

            print(f'{self.home_name()} - {self.away_name()}:- \n')

            print(goal.text)
        
        except IndexError:
        
            print(f'Match upcoming \n')

   


In [6]:
import numpy as np    
########## Making csv/Excel files ############

class Visualization(HomeTeam, AwayTeam):
    
    def __init__(self, x):
        
        super().__init__(x)
        
    def home_file(self):
        
   
        home_goal = self.home_goal() 
    
        home_goal_no = self.goal_no_home()
        
        home_assist = self.home_assist()

        
        if len(home_goal) > 0:   # filtering red cards from home team
            for i in range(len(home_goal)): 
                f = re.compile(r'(Red Card)')
                l = f.findall(home_goal[i])
                if l:
                    home_goal[i] = np.nan
                    home_goal_no[i] = np.nan
        
        if len(home_goal) == 0:    # for no score home team
            home_goal = [np.nan]
            home_assist = [np.nan]
            home_goal_no = [np.nan]
                 
            
    
        df_home_goal = pd.DataFrame({self.home_name()+' - goal': home_goal,
                                     'Goals_no': home_goal_no})
                                    
        
        df_home_assist = pd.DataFrame({self.home_name()+' - assists': home_assist})
        
        df_home = pd.concat([df_home_goal,
                             df_home_assist], axis=1)
        
        return df_home
        
    def away_file(self):
             
        away_goal = self.away_goal()
        
        away_assist = self.away_assist()
        
        away_goal_no = self.goal_no_away()
        
        
        if len(away_goal) > 0:   # filtering red cards from away team
            for i in range(len(away_goal)): 
                f = re.compile(r'(Red Card)')
                l = f.findall(away_goal[i])
                if l:
                    away_goal[i] = np.nan
                    away_goal_no[i] = np.nan

        
        if len(away_goal) == 0:    # for no score away team
            away_goal = [np.nan]
            away_assist = [np.nan]
            away_goal_no = [np.nan]
                        
            
            
        
        df_away_goal = pd.DataFrame({self.away_name()+' - goal': away_goal,
                                     'Goals_no': away_goal_no})
        
        df_away_assist = pd.DataFrame(away_assist,
                                      columns=[self.away_name()+' - assists'])
        
        df_away = pd.concat([df_away_goal,
                             df_away_assist], axis=1)
        
        return df_away
    

In [62]:
# output match result epl offc site
id = 59005
score = MatchResult(id)
score.result()
saving = Visualization(id)

df_h = saving.home_file()
df_a = saving.away_file()

# adding date time index in dataframe

#df['date'] = [today for i in range(df.shape[0])]
#df.set_index('date', inplace=True)


df_a

# Updating files for each team(due)

#df.to_csv('Chelsea.csv')

West Ham United - Manchester United:- 

1-3


,Manchester United - goal,Goals_no,Manchester United - assists
0,Goal - Paul Pogba,1,Bruno Fernandes
1,Goal - Mason Greenwood,1,Alex Telles
2,Goal - Marcus Rashford,1,Juan Mata


In [52]:
# Getting all matches in this week and id code epl offc site 

def completed_match():
    link = 'https://www.premierleague.com/'

    r = requests.get(link)

    soup = bs(r.content, 'html.parser')

    number = soup.select('a.matchAbridged.form.embeddableMatchContainer')
    
    Team_code = []
    for tag in number:

        text = tag.get('href')
        c = tag.select('span.teamName')
        T = []
        for team_1 in c:

            team = team_1.abbr.get('title')
            T.append(team)



        pattern = re.compile(r'\d+')
        code = pattern.findall(text)
        T.append(code)

        Team_code.append(T)
        return Team_code
    

CompletedMatch = completed_match()

if not CompletedMatch:
    print('No completed Match updated in site(this week) \n visit:- https://www.premierleague.com/')
else:
    completed_match()

No completed Match updated in site(this week) 
 visit:- https://www.premierleague.com/


In [60]:
# weekly fixtures matches with code premire league official site

def match_date(id):

    url = 'https://www.premierleague.com/match/{}'.format(id)

    res = requests.get(url)

    soup = bs(res.text, "lxml")
    
    data = json.loads(soup.find("div", class_="mcTabsContainer")["data-fixture"])

    return data['provisionalKickoff']['label']

def weekly_fixtures():
    
    link = 'https://www.premierleague.com/home'

    r = requests.get(link)
    soup = bs(r.content, 'html.parser')

    fixtures = soup.select('a.matchAbridged.embeddableMatchContainer')
    
    Team_code = []
    date = []
    
    for fixture in fixtures:
        text = fixture.get('href')
        c = fixture.select('span.teamName')
        T = {}
        for i in range(len(c)):
            if i == 0:
                team = c[i].abbr.get('title')
                T['Team_1'] = team
            elif i == 1:
                team = c[i].abbr.get('title')
                T['Team_2'] = team

        pattern = re.compile(r'\d+')
        code = pattern.findall(text)
        T['id']=code[0]
        
        id = code[0]
        a = match_date(id)
        T['date'] = a
        Team_code.append(T)
 
    return Team_code

fixtures = weekly_fixtures()

    
# if not fixtures:
#     print('No match fixtures in this week \n visit:- https://www.premierleague.com/')
# else:
#     print(fixtures)
df = pd.DataFrame(fixtures[2], index=[0])    
df

,Team_1,Team_2,id,date
0,West Ham United,Manchester United,59005,"Sat 5 Dec 2020, 17:30 GMT"


# This is from ESPN


In [65]:
# to get all scorer and assist for each team espn
import requests
import pandas as pd

from bs4 import BeautifulSoup

def stat(code):
    url = 'https://www.espn.in/football/team/stats/_/id/{}/league/ENG.1'.format(code)
    df = pd.read_html(url)
    return df
def goal_stat(code):
    goal = stat(code)[0]
    return goal
def assist_stat(code):
    assist = stat(code)[1]
    return assist

id = 360
df_goal = goal_stat(id)
df_goal['Goal Percent'] = (df_goal['G']/df_goal['P'])*100
#df
df_assist = assist_stat(id)
df_assist['Assist Percent'] = (df_assist['A']/df_assist['P'])*100
df_goal

,RK,Name,P,G,Goal Percent
0,1.0,Bruno Fernandes,10,7,70.000000
1,2.0,Marcus Rashford,10,3,30.000000
2,NaN,Edinson Cavani,6,3,50.000000
3,4.0,Harry Maguire,10,1,10.000000
4,NaN,Aaron Wan-Bissaka,9,1,11.111111
5,NaN,Donny van de Beek,8,1,12.500000
6,NaN,Paul Pogba,8,1,12.500000
7,NaN,Mason Greenwood,7,1,14.285714
8,9.0,Victor Lindelöf,9,0,0.000000
9,NaN,David de Gea,9,0,0.000000


In [66]:
df_assist

,RK,Name,P,A,Assist Percent
0,1.0,Marcus Rashford,10,4,40.000000
1,NaN,Bruno Fernandes,10,4,40.000000
2,3.0,Juan Mata,5,2,40.000000
3,4.0,Luke Shaw,7,1,14.285714
4,NaN,Edinson Cavani,6,1,16.666667
5,NaN,Alex Telles,3,1,33.333333
6,7.0,Harry Maguire,10,0,0.000000
7,NaN,Aaron Wan-Bissaka,9,0,0.000000
8,NaN,Victor Lindelöf,9,0,0.000000
9,NaN,David de Gea,9,0,0.000000


In [12]:
# team code for goal number and assists espn
import requests
import pandas as pd
from bs4 import BeautifulSoup

r = requests.get('https://www.espn.in/football/league/_/name/eng.1/')
soup = BeautifulSoup(r.content, 'html.parser')
teams = soup.select('tr a')
for team in teams:
    link = team.get('href')
    pattern = re.compile(r'\d+')
    match = pattern.findall(link)
    print(team.text+','+match[0])

Tottenham Hotspur,367
Liverpool,364
Chelsea,363
Leicester City,375
West Ham United,371
Southampton,376
Wolverhampton Wanderers,380
Everton,368
Manchester United,360
Aston Villa,362
Manchester City,382
Leeds United,357
Newcastle United,361
Arsenal,359
Crystal Palace,384
Brighton & Hove Albion,331
Fulham,370
West Bromwich Albion,383
Burnley,379
Sheffield United,398


# This is Premire League standings

In [63]:
# premire leaugue standings

url = 'https://www.premierleague.com/home'
def standing():
    df = pd.read_html(url)
    return df[0]
standing()

,Pos,Club,Pl,GD,Pts
0,1 Previous Position 3,Chelsea,11,14,22
1,2 Previous Position 1,Spurs,10,12,21
2,3 Previous Position 2,Liverpool,10,5,21
3,4 Previous Position 9,Man Utd,10,2,19
4,5 Previous Position 11,Man City,10,6,18
5,6 Previous Position 4,Leicester,10,5,18
6,7 Previous Position 5,West Ham,11,4,17
7,8 Previous Position 6,Southampton,10,3,17
8,9 Previous Position 8,Everton,11,2,17
9,10 Previous Position 7,Wolves,10,0,17


In [14]:
# finding fantasy stats (incomplete)


12/05/2020 18:10:04 GMT
2011-01-21 02:37:21+00:00
2011-01-20 21:37:21-05:00
